1. Datenaufbereitung

In [3]:
import pandas as pd

# Datei laden
file_path = "monthly_hourly_load_values_2022_DE.csv"

# Datei mit korrektem Trennzeichen laden
df = pd.read_csv(file_path, delimiter=';')

# Zeitspalte umwandeln
df["DateUTC"] = pd.to_datetime(df["DateUTC"], format="%d/%m/%Y %H:%M")

# Zeit als Index setzen
df.set_index("DateUTC", inplace=True)

# Ersten Einträge anzeigen
df.head()

,MeasureItem,DateShort,TimeFrom,TimeTo,CountryCode,Cov_ratio,Value,Value_ScaleTo100,CreateDate
DateUTC,,,,,,,,,
2022-01-01 00:00:00,Monthly Hourly Load Values,01/01/2022,00:00,01:00,DE,100,41535.7650,41535.7650,07/06/2023 15:31
2022-01-01 01:00:00,Monthly Hourly Load Values,01/01/2022,01:00,02:00,DE,100,40480.9050,40480.9050,07/06/2023 15:31
2022-01-01 02:00:00,Monthly Hourly Load Values,01/01/2022,02:00,03:00,DE,100,39563.9150,39563.9150,07/06/2023 15:31
2022-01-01 03:00:00,Monthly Hourly Load Values,01/01/2022,03:00,04:00,DE,100,39338.1950,39338.1950,07/06/2023 15:31
2022-01-01 04:00:00,Monthly Hourly Load Values,01/01/2022,04:00,05:00,DE,100,38826.6425,38826.6425,07/06/2023 15:31


2 Algorithmus: Long Short-Term Memory (LSTM)

In [4]:

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# 1. Nur Zielwert extrahieren
data = df["Value"].values.reshape(-1, 1)

# 2. Normalisieren
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)


# 3. Sequenzen erstellen
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length])
        y.append(data[i + seq_length])
    return np.array(X), np.array(y)


SEQ_LEN = 24  # Eingabefenster (z. B. 24 Stunden)
X, y = create_sequences(scaled_data, SEQ_LEN)

# 4. Training/Test Split
split = int(len(X) * 0.8)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# 5. LSTM-Modell erstellen und trainieren
model = Sequential()
model.add(LSTM(64, activation='relu', input_shape=(SEQ_LEN, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

# 6. Vorhersage und Evaluation
y_pred = model.predict(X_test)
y_pred_rescaled = scaler.inverse_transform(y_pred)
y_test_rescaled = scaler.inverse_transform(y_test)

mse = mean_squared_error(y_test_rescaled, y_pred_rescaled)
mae = mean_absolute_error(y_test_rescaled, y_pred_rescaled)

print(f"✅ Evaluation abgeschlossen: MSE = {mse:.2f}, MAE = {mae:.2f}")

Epoch 1/10


/opt/conda/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


197/197 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 0.0400 - val_loss: 0.0063
Epoch 2/10
197/197 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.0051 - val_loss: 0.0026
Epoch 3/10
197/197 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.0025 - val_loss: 0.0019
Epoch 4/10
197/197 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.0016 - val_loss: 0.0013
Epoch 5/10
197/197 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0012 - val_loss: 0.0023
Epoch 6/10
197/197 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.0014 - val_loss: 0.0014
Epoch 7/10
197/197 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0012 - val_loss: 0.0011
Epoch 8/10
197/197 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 9.0902e-04 - val_loss: 0.0012
Epoch 9/10
197/197 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 8.9423e-04 - val_loss: 8.0122e-04
Epoch 10/10
197/197 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 8.1131e-04 - val_loss: 8.5081e-04
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
✅ Evaluation abgeschlossen: MSE = 1882569.66, MAE = 988.51


In [6]:
model.save("lstm_model.keras")

3 Gated Recurrent Unit (GRU)

In [9]:

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense

# 1. Zielwert extrahieren
data = df["Value"].values.reshape(-1, 1)

# 2. Normalisieren
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)


# 3. Sequenzen erstellen
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length])
        y.append(data[i + seq_length])
    return np.array(X), np.array(y)


SEQ_LEN = 24  # Eingabefenster (24 Stunden)
X, y = create_sequences(scaled_data, SEQ_LEN)

# 4. Trainings- und Testdaten aufteilen
split = int(len(X) * 0.8)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# 5. GRU-Modell erstellen und trainieren
model = Sequential()
model.add(GRU(64, activation='relu', input_shape=(SEQ_LEN, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

# 6. Vorhersagen und Evaluation
y_pred = model.predict(X_test)
y_pred_rescaled = scaler.inverse_transform(y_pred)
y_test_rescaled = scaler.inverse_transform(y_test)

mse = mean_squared_error(y_test_rescaled, y_pred_rescaled)
mae = mean_absolute_error(y_test_rescaled, y_pred_rescaled)

print(f"✅ GRU Evaluation abgeschlossen: MSE = {mse:.2f}, MAE = {mae:.2f}")

Epoch 1/10


/opt/conda/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


197/197 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 0.0602 - val_loss: 0.0055
Epoch 2/10
197/197 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 0.0038 - val_loss: 0.0025
Epoch 3/10
197/197 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 0.0020 - val_loss: 0.0022
Epoch 4/10
197/197 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.0015 - val_loss: 0.0014
Epoch 5/10
197/197 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.0014 - val_loss: 0.0015
Epoch 6/10
197/197 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.0012 - val_loss: 0.0012
Epoch 7/10
197/197 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 9.5998e-04 - val_loss: 0.0013
Epoch 8/10
197/197 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 9.6436e-04 - val_loss: 9.4390e-04
Epoch 9/10
197/197 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 8.3657e-04 - val_loss: 9.1642e-04
Epoch 10/10
197/197 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 7.8834e-04 - val_loss: 8.0785e-04
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
✅ GRU Evaluation abgeschlossen: MSE = 1453956.31, MAE = 887.63


In [11]:
model.save("gru_model.keras")

In [21]:
import streamlit as st
import pandas as pd
import numpy as np
import shap
import lime.lime_tabular
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import load_model

# ------------------------------
# Parameter & Konstanten
# ------------------------------
SEQ_LEN = 24

# ------------------------------
# Daten vorbereiten
# ------------------------------
df = pd.read_csv("monthly_hourly_load_values_2022_DE.csv", delimiter=";", parse_dates=["TimeTo"], index_col="TimeTo")
scaler = MinMaxScaler()
scaler.fit(df[["Value"]].values)

# ------------------------------
# Modelle laden (.keras Format)
# ------------------------------
lstm_model = load_model("lstm_model.keras")
gru_model = load_model("gru_model.keras")


# ------------------------------
# Dummy Input erstellen (ersetzbar durch Live-Daten)
# ------------------------------
def get_dummy_input():
    values = np.linspace(20000, 80000, SEQ_LEN).reshape(-1, 1)
    scaled = scaler.transform(values).reshape(1, SEQ_LEN, 1).astype(np.float32)
    return scaled


# ------------------------------
# LIME-Erklärung generieren
# ------------------------------
def explain_with_lime(model, input_data):
    explainer = lime.lime_tabular.LimeTabularExplainer(
        training_data=np.zeros((10, SEQ_LEN)),
        mode="regression"
    )
    explanation = explainer.explain_instance(
        input_data.flatten(),
        lambda x: model.predict(x.reshape(-1, SEQ_LEN, 1))
    )
    return explanation.as_list()


# ------------------------------
# SHAP-Erklärung generieren
# ------------------------------
def explain_with_shap(model, input_data):
    background = np.zeros((10, SEQ_LEN)).astype(np.float32)
    masker = shap.maskers.Independent(background)
    explainer = shap.Explainer(lambda x: model.predict(x.reshape(-1, SEQ_LEN, 1)), masker)
    shap_values = explainer(input_data.reshape(1, SEQ_LEN))
    return shap_values.values.tolist()


# ------------------------------
# Streamlit UI
# ------------------------------
st.title("XAI Vergleich: LSTM & GRU mit LIME & SHAP")

input_data = get_dummy_input()

# Vier Tabs
tab1, tab2, tab3, tab4 = st.tabs([
    "LSTM mit LIME", "LSTM mit SHAP", "GRU mit LIME", "GRU mit SHAP"
])

# Tab 1 – LSTM + LIME
with tab1:
    st.subheader("Long Short-Term Memory (LSTM) using explainer LIME")
    prediction = lstm_model.predict(input_data)
    lime_exp = explain_with_lime(lstm_model, input_data)
    st.write("Vorhersage:", prediction.flatten()[0])
    st.write("LIME Erklärung:", lime_exp)

# Tab 2 – LSTM + SHAP
with tab2:
    st.subheader("Long Short-Term Memory (LSTM) using explainer SHAP")
    prediction = lstm_model.predict(input_data)
    shap_exp = explain_with_shap(lstm_model, input_data)
    st.write("Vorhersage:", prediction.flatten()[0])
    st.write("SHAP Werte:", shap_exp)

# Tab 3 – GRU + LIME
with tab3:
    st.subheader("Gated Recurrent Unit (GRU) using explainer LIME")
    prediction = gru_model.predict(input_data)
    lime_exp = explain_with_lime(gru_model, input_data)
    st.write("Vorhersage:", prediction.flatten()[0])
    st.write("LIME Erklärung:", lime_exp)

# Tab 4 – GRU + SHAP
with tab4:
    st.subheader("Gated Recurrent Unit (GRU) using explainer SHAP")
    prediction = gru_model.predict(input_data)
    shap_exp = explain_with_shap(gru_model, input_data)
    st.write("Vorhersage:", prediction.flatten()[0])
    st.write("SHAP Werte:", shap_exp)

/tmp/ipykernel_149/3703357769.py:17: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv("monthly_hourly_load_values_2022_DE.csv", delimiter=";", parse_dates=["TimeTo"], index_col="TimeTo")
/opt/conda/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 7 variables whereas the saved optimizer has 12 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
2025-03-26 14:41:44.366 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-26 14:41:44.367 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-26 14:41:44.368 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-0

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step
157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


2025-03-26 14:41:45.403 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-26 14:41:45.404 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-26 14:41:45.404 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-26 14:41:45.405 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-26 14:41:45.406 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-26 14:41:45.407 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-26 14:41:45.407 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-26 14:41:45.408 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


2025-03-26 14:41:46.583 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-26 14:41:46.584 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-26 14:41:46.584 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-26 14:41:46.585 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-26 14:41:46.585 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-26 14:41:46.586 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-26 14:41:46.586 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-26 14:41:46.587 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step
157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step


2025-03-26 14:41:47.654 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-26 14:41:47.655 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-26 14:41:47.656 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-26 14:41:47.656 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-26 14:41:47.657 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-26 14:41:47.657 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-26 14:41:47.658 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-26 14:41:47.658 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


2025-03-26 14:41:48.904 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-26 14:41:48.905 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-26 14:41:48.905 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-26 14:41:48.906 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-26 14:41:48.906 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-26 14:41:48.907 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-26 14:41:48.907 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-26 14:41:48.908 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar